In [16]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, TFBertForMaskedLM
import pandas as pd
import numpy as np

In [14]:
dat = pd.read_csv("/Users/samknowles/Downloads/skincap_vqa.csv")
df = pd.read_csv("hf://datasets/Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning/combined_data.csv")

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

qa_pairs = [(df.loc[i,"prompt"], df.loc[i,"response"]) for i in range(len(df))]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [40]:
def prepare_qa_data(qa_pairs):

    texts = [f"Question: {q} Answer: {a}" for q, a in qa_pairs]

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="tf"
    )

    return inputs

def mask_tokens(inputs, tokenizer):

    inputs['labels'] = inputs['input_ids']

    probability_matrix = np.random.rand(*inputs['input_ids'].shape)

    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in inputs['input_ids'].numpy()
    ]
    probability_matrix = np.where(np.array(special_tokens_mask, dtype=bool), 0.0, probability_matrix)

    masked_indices = probability_matrix < 0.15
    inputs['input_ids'] = np.where(masked_indices, tokenizer.mask_token_id, inputs['input_ids'])

    inputs['input_ids'] = tf.convert_to_tensor(inputs['input_ids'])
    inputs['labelss'] = tf.convert_to_tensor(inputs['labels'])


    return inputs

inputs = prepare_qa_data(qa_pairs)

masked_inputs = mask_tokens(inputs, tokenizer)

optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)

model.compile(optimizer=optimizer)

model.fit(masked_inputs['input_ids'], masked_inputs['labels'], epochs=5, batch_size=2)

Epoch 1/5
730/730 [==============================] - 1043s 1s/step - loss: 0.2156
Epoch 2/5
730/730 [==============================] - 1083s 1s/step - loss: 0.0838
Epoch 3/5
730/730 [==============================] - 1120s 2s/step - loss: 0.0457
Epoch 4/5
730/730 [==============================] - 1138s 2s/step - loss: 0.0293
Epoch 5/5
730/730 [==============================] - 1182s 2s/step - loss: 0.0193


In [39]:
masked_inputs['input_ids'][0]

<tf.Tensor: shape=(243,), dtype=int32, numpy=
array([  103,  3160,  1024,  2054,   103,   103, 11069,  6190,   103,
        2054,  2024,  2049,  2691,  8030,  1029,   103,  1024,  8827,
       11069,  6190,  2003,  1037, 11888,  8285,  5714,   103,  2063,
        4650,  2008,  3463,  1999,  1996,  2058, 21572, 16256,  1997,
         103,  4442,  1012,  2023,  2058, 21572, 16256,  5260,  2000,
         103,  1997,  4317,  1010,  2417,  3096,  3139,  2007, 21666,
        9539,  1012,   103,  8030,  2421,  2417, 13864,  1997,  3096,
        3139,  2007,  4317,  1010, 21666,  9539,  1010,  2235, 25169,
        7516,   103,  4141,  2464,  1999,  2336,  1007,   103,  4318,
        1998,  9630,  3096,  2008,  2089, 19501,   103,  2009,  8450,
        1010,  5255,  1010,  2030, 14699,   103,  1010,  4317,  6675,
        1010, 25895,  1010,   103,  5526,  2094, 10063,  1010,  1998,
       13408,  1998, 10551, 17651,  1012,   103,   103,   103,   103,
         103,   103,   103,   103,   103,   

In [41]:
import os
os.getcwd()

'/Users/samknowles/Desktop/Projects/iLab/Skintelligence/Notebooks'

In [42]:
save_directory = "./my_pretrained_bert"

In [43]:
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

('./my_pretrained_bert/tokenizer_config.json',
 './my_pretrained_bert/special_tokens_map.json',
 './my_pretrained_bert/vocab.txt',
 './my_pretrained_bert/added_tokens.json')